This article discussed a couple of subsetting and shrinkage methods:
* Best Subset Regression iterates over all possible feature combination to select the best one;
* Ridge Regression penalizes the squared coefficient values (L2 penalty) enforcing them to be small;
* LASSO penalizes the absolute values of the coefficients (L1 penalty) which can force some of them to be exactly zero;
* Elastic Net combines the L1 and L2 penalties, enjoying the best of Ridge and Lasso;
* Least Angle Regression fits in between subsetting and shrinkage: it works iteratively, adding “some part” of one of the features at each step;
* Principal Components Regression performs PCA to squeeze the original features into a small subset of new features and then uses those as predictors;
* Partial Least Squares also summarizes original features into a smaller subset of new ones, but unlike PCR, it also makes use of the targets to construct them.

In [4]:
# Import necessary modules and set options
import pandas as pd
import numpy as np
import itertools

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LarsCV
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

# Load data
data = pd.read_csv("prostate.data.csv", sep = "\t")
print(data.head())

# Train-test split
y_train = np.array(data[data.train == "T"]['lpsa'])
y_test = np.array(data[data.train == "F"]['lpsa'])
X_train = np.array(data[data.train == "T"].drop(['lpsa', 'train'], axis=1))
X_test = np.array(data[data.train == "F"].drop(['lpsa', 'train'], axis=1))

   Unnamed: 0    lcavol   lweight  age      lbph  svi       lcp  gleason  \
0           1 -0.579818  2.769459   50 -1.386294    0 -1.386294        6   
1           2 -0.994252  3.319626   58 -1.386294    0 -1.386294        6   
2           3 -0.510826  2.691243   74 -1.386294    0 -1.386294        7   
3           4 -1.203973  3.282789   58 -1.386294    0 -1.386294        6   
4           5  0.751416  3.432373   62 -1.386294    0 -1.386294        6   

   pgg45      lpsa train  
0      0 -0.430783     T  
1      0 -0.162519     T  
2     20 -0.162519     T  
3      0 -0.162519     T  
4      0  0.371564     T  


In [5]:

linreg_model = LinearRegression(normalize=True).fit(X_train, y_train)
linreg_prediction = linreg_model.predict(X_test)
linreg_mae = np.mean(np.abs(y_test - linreg_prediction))
linreg_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1],
        np.round(np.concatenate((linreg_model.intercept_, linreg_model.coef_),
        axis=None), 3))
)

print('Linear Regression MAE: {}'.format(np.round(linreg_mae, 3)))
print('Linear Regression coefficients:')
linreg_coefs

Linear Regression MAE: 0.24
Linear Regression coefficients:


{'Intercept': 1.014,
 'Unnamed: 0': 0.034,
 'lcavol': 0.145,
 'lweight': 0.056,
 'age': -0.007,
 'lbph': 0.046,
 'svi': 0.066,
 'lcp': -0.052,
 'gleason': -0.042,
 'pgg45': 0.003}

In [6]:

results = pd.DataFrame(columns=['num_features', 'features', 'MAE'])

# Loop over all possible numbers of features to be included
for k in range(1, X_train.shape[1] + 1):
    # Loop over all possible subsets of size k
    for subset in itertools.combinations(range(X_train.shape[1]), k):
        subset = list(subset)
        linreg_model = LinearRegression(normalize=True).fit(X_train[:, subset], y_train)
        linreg_prediction = linreg_model.predict(X_test[:, subset])
        linreg_mae = np.mean(np.abs(y_test - linreg_prediction))
        results = results.append(pd.DataFrame([{'num_features': k,
                                                'features': subset,
                                                'MAE': linreg_mae}]))

# Inspect best combinations
results = results.sort_values('MAE').reset_index()
print(results.head())

# Fit best model
best_subset_model = LinearRegression(normalize=True).fit(X_train[:, results['features'][0]], y_train)
best_subset_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1],
        np.round(np.concatenate((best_subset_model.intercept_, best_subset_model.coef_), axis=None), 3))
)

print('Best Subset Regression MAE: {}'.format(np.round(results['MAE'][0], 3)))
print('Best Subset Regression coefficients:')
best_subset_coefs

   index num_features            features       MAE
0      0            4        [0, 2, 3, 4]  0.221212
1      0            3           [0, 3, 4]  0.222467
2      0            3           [0, 2, 3]  0.222898
3      0            6  [0, 1, 2, 3, 4, 6]  0.224248
4      0            5     [0, 1, 3, 4, 6]  0.224347
Best Subset Regression MAE: 0.221
Best Subset Regression coefficients:


{'Intercept': 0.607,
 'Unnamed: 0': 0.039,
 'lcavol': 0.025,
 'lweight': -0.002,
 'age': 0.027}

In [7]:
ridge_cv = RidgeCV(normalize=True, alphas=np.logspace(-10, 1, 400))
ridge_model = ridge_cv.fit(X_train, y_train)
ridge_prediction = ridge_model.predict(X_test)
ridge_mae = np.mean(np.abs(y_test - ridge_prediction))
ridge_coefs = dict(
    zip(['Intercept'] + data.columns.tolist()[:-1],
        np.round(np.concatenate((ridge_model.intercept_, ridge_model.coef_),
                                axis=None), 3))
)

print('Ridge Regression MAE: {}'.format(np.round(ridge_mae, 3)))
print('Ridge Regression coefficients:')
ridge_coefs

Ridge Regression MAE: 0.239
Ridge Regression coefficients:


{'Intercept': 0.948,
 'Unnamed: 0': 0.033,
 'lcavol': 0.154,
 'lweight': 0.076,
 'age': -0.007,
 'lbph': 0.049,
 'svi': 0.088,
 'lcp': -0.051,
 'gleason': -0.035,
 'pgg45': 0.003}